In [1]:
import json
import os
from os import path
import numpy as np
import matplotlib.pyplot as plt
import shapely.wkt
from cv2 import fillPoly, imwrite
from PIL import Image
from tqdm import tqdm

In [2]:
DMG_CLASSES = {"no-damage":0,"minor-damage":1,"major-damage":2,"destroyed":3}
DMG_NAMES = {0:"no-damage",1:"minor-damage",2:"major-damage",3:"destroyed"}

IMG_FOLDER = [
            './data/mexico/pre'            
             ]
LABEL_FOLDER = [
            './data/mexico/labels/'
                ]

MASK_FOLDER = [
                './data/mexico/mask/'
                ]

In [3]:
def polygons_to_mask_pre(PATH,out_type="many"):
    """
    :param PATH: A path to a JSON with Polygons
    :param out_type: changes output type
    :returns: If output type is one, returns a 1024x1024 np.array with values 1-4 
    corresponding to building dmg scales 0-4, where if polygons overlapped in the 
    input, the maximum dmg was used. If output type is many returns a 1024x1024x4 
    np.array with values 0 or 1 corresponding to no building vs. building of dmg 
    type x in channel x.
    """
    JSON = json.load(open(PATH))
    polygons = []
    for polygon in JSON['features']["xy"]:
        if (polygon['properties']['feature_type'] == 'building'):
            coords = list(shapely.geometry.mapping(shapely.wkt.loads(polygon['wkt']))['coordinates'][0])
            polygons.append((1,np.array(coords, np.int32)))
    size = (1024,1024,5)
    mask_img = np.zeros(size, np.uint8)

    if out_type == "many":
        for poly in polygons:
            blank =  np.zeros((1024,1024), np.uint8)
            fillPoly(blank, [poly[1]], color=1)
            mask_img[:,:,poly[0]+1] = np.maximum(mask_img[:,:,poly[0]+1],blank)
        mask_img[:,:,0] = np.ones((1024,1024)) - np.maximum(np.maximum(np.maximum(mask_img[:,:,1],mask_img[:,:,2]),mask_img[:,:,3]),mask_img[:,:,4])
        return mask_img

    else:
        for poly in polygons:
            blank =  np.zeros((1024,1024), np.uint8)
            fillPoly(blank, [poly[1]], color=poly[0]+1)
            mask_img[:,:,poly[0]+1] = np.maximum(mask_img[:,:,poly[0]+1],blank)
        mask_all = np.maximum(np.maximum(np.maximum(mask_img[:,:,1],mask_img[:,:,2]),mask_img[:,:,3]),mask_img[:,:,4])
        return mask_all

In [4]:
def polygons_to_mask_post(PATH,out_type="many"):
    """
    :param PATH: A path to a JSON with Polygons
    :param out_type: changes output type
    :returns: If output type is one, returns a 1024x1024 np.array with values 1-4 
    corresponding to building dmg scales 0-4, where if polygons overlapped in the 
    input, the maximum dmg was used. If output type is many returns a 1024x1024x4 
    np.array with values 0 or 1 corresponding to no building vs. building of dmg 
    type x in channel x.
    """
    JSON = json.load(open(PATH))
    polygons = []
    for polygon in JSON['features']["xy"]:
        if (polygon['properties']['subtype'] != 'un-classified'):
            dmgtype = DMG_CLASSES[polygon['properties']['subtype']]
            coords = list(shapely.geometry.mapping(shapely.wkt.loads(polygon['wkt']))['coordinates'][0])
            polygons.append((dmgtype,np.array(coords, np.int32)))
    size = (1024,1024,5)
    mask_img = np.zeros(size, np.uint8)

    if out_type == "many":
        for poly in polygons:
            blank =  np.zeros((1024,1024), np.uint8)
            fillPoly(blank, [poly[1]], color=1)
            mask_img[:,:,poly[0]+1] = np.maximum(mask_img[:,:,poly[0]+1],blank)
        mask_img[:,:,0] = np.ones((1024,1024)) - np.maximum(np.maximum(np.maximum(mask_img[:,:,1],mask_img[:,:,2]),mask_img[:,:,3]),mask_img[:,:,4])
        return mask_img

    else:
        for poly in polygons:
            blank =  np.zeros((1024,1024), np.uint8)
            fillPoly(blank, [poly[1]], color=poly[0]+1)
            mask_img[:,:,poly[0]+1] = np.maximum(mask_img[:,:,poly[0]+1],blank)
        mask_all = np.maximum(np.maximum(np.maximum(mask_img[:,:,1],mask_img[:,:,2]),mask_img[:,:,3]),mask_img[:,:,4])
        return mask_all

In [5]:
def create_mask_png_post(IN_FOLDER,IN_FILE,OUT_FOLDER):
    """
    :param IN_FOLDER: A path to the input folder with jsons
    :param IN_FILE: name of input json
    :param OUT_FOLDER: Path to output folder for mask pngs
    """
    mask_all = polygons_to_mask_post(IN_FOLDER+IN_FILE,out_type="polygons_to_mask")
    mask = Image.fromarray(mask_all)
    colorize_mask_(mask)
    mask.save(OUT_FOLDER+IN_FILE[:-5]+"_mask.png")

In [6]:
def create_mask_png_pre(IN_FOLDER,IN_FILE,OUT_FOLDER):
    """
    :param IN_FOLDER: A path to the input folder with jsons
    :param IN_FILE: name of input json
    :param OUT_FOLDER: Path to output folder for mask pngs
    """
    mask_all = polygons_to_mask_pre(IN_FOLDER+IN_FILE,out_type="polygons_to_mask")
    mask = Image.fromarray(mask_all)
    colorize_mask_(mask)
    mask.save(OUT_FOLDER+IN_FILE[:-5]+"_mask.png")

In [7]:
def colorize_mask_(mask, color_map=None):
    """
    Attaches a color palette to a PIL image. So long as the image is saved as a PNG, it will render visibly using the
    provided color map.
    :param mask: PIL image whose values are only 0 to 4 inclusive
    :param color_map: np.ndarray or list of 3-tuples with 5 rows
    :return:
    """
    color_map = color_map or np.array([(0, 0, 0),  # 0=background --> black
                                       (255, 255,255),  # no damage (or just 'building' for localization) --> green
                                       (255, 255, 255),  # minor damage --> yellow
                                       (255, 255, 255),  # major damage --> orange
                                       (255, 255, 255),  # destroyed --> red
                                       ])
    assert color_map.shape == (5, 3)
    mask.putpalette(color_map.astype(np.uint8))
    return None

In [8]:
for folder in MASK_FOLDER:
    os.makedirs(folder, exist_ok=True)

In [9]:
keywords = ["post"]
for file in tqdm(os.listdir(LABEL_FOLDER[0])):
    if file.endswith(".json"):
        filename = os.path.splitext(file)[0]
        filename_fragments = filename.split("_")
        samekeywords = list(set(filename_fragments) & set(keywords)) #to select only the post jsons
        if len(samekeywords) == len(keywords):
            create_mask_png_post(LABEL_FOLDER[0],file,MASK_FOLDER[0])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 242/242 [00:35<00:00,  6.75it/s]


In [10]:
keywords = ["pre"]
for file in tqdm(os.listdir(LABEL_FOLDER[0])):
    if file.endswith(".json"):
        filename = os.path.splitext(file)[0]
        filename_fragments = filename.split("_")
        samekeywords = list(set(filename_fragments) & set(keywords)) #to select only the pre jsons
        if len(samekeywords) == len(keywords):
            create_mask_png_pre(LABEL_FOLDER[0],file,MASK_FOLDER[0])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 242/242 [00:36<00:00,  6.61it/s]
